In [1]:
import h5py
import os
import numpy as np
import xlsxwriter

cwd = os.getcwd()
os.chdir('C:\\Users\\amorf\\Documents\\Oxford\\Python')
from general_funcs import *
from ARPES_funcs import *
from RvsT_funcs04 import *
os.chdir(cwd)

In [2]:
from datetime import date
def get_date():
    tday = date.today()
    y, m, d = tday.year, tday.month, tday.day
    return str(d)+'/'+str(m)+'/'+str(y)

In [3]:
def reduce_SC(SC):
    newSC = ''
    bits = SC.split(' ')
    for bit in bits:
        try:
            num = float(bit)
            red_num = sigfig(num, 4)
            newSC += str(red_num) + ' '
        except:
            newSC += bit + ' '
    return newSC

def single_sigfig(x, sf):
    if x in [np.inf, np.nan, 0]:
        return x
    else:
        return np.round(x, sf-math.floor(math.log10(abs(x)))-1)
              
def sigfig(X, sf):
    try:
        Xs = []
        for x in X:
            Xs.append(single_sigfig(x, sf))
        return np.asarray(Xs)
    except:
        return single_sigfig(X, sf)

In [18]:
sample = 'Am18P_2'
os.chdir('\\\cmdaq3.physics.ox.ac.uk\\CMDAQ_RestOfCMGroups\\ColdeaAGroup\\Data2024\\ARPES_Jan24\\Am18P_2')

In [19]:
#Open my Excel file, if the file exists it just opens that one
wb = xlsxwriter.Workbook(sample+'_Logbook.xlsx')

#Add a sheet
ws = wb.add_worksheet('Sample1')

# Create self format
bold = wb.add_format()
bold.set_bold()
bold.set_align('center')

centering = wb.add_format()
centering.set_align('center')

# Sign the Logbook
ws.write('B1', 'Logbook made by Archie Morfoot on ' + get_date())

# Add column headers
ws.write('A2', 'Fileno')
ws.write('B2', 'Scan command')
ws.write('C2', 'Dim')
ws.write('D2', 'Mode')
ws.write('E2', 'Pol')
ws.write('F2', 'Cryo T (K)')
ws.write('G2', 'Ph En (eV)')
ws.write('H2', 'sapolar')
ws.write('I2', 'deflector_x')
ws.write('J2', 'saazimuth')
ws.write('K2', 'satilt')
ws.write('L2', 'sax')
ws.write('M2', 'say')
ws.write('N2', 'saz')
ws.write('O2', 'Pass E (eV)')
ws.write('P2', 'Ex Slit (mm)')

rowIndex = 3
manipulator_keys = ['sapolar', 'saazimuth', 'satilt', 'sax', 'say', 'saz']

for fname in os.listdir():
    if fname[-4:] == '.nxs':
        with h5py.File(fname, 'r') as f:
            
            # Define all keys in entry1
            entry_keys = [*f['entry1'].keys()]
            
            # Need to add a condition for if the camera or other is used
            if 'analyser' in entry_keys:
                # Find dimensionality
                if len(set(['sapolar', 'deflector_x']).intersection(set(f['entry1/analyser/'].keys()))) != 0:
                    dim = 3
                else:
                    dim = 2

                # Scan command
                ScanCommand = str(np.squeeze(f['entry1/scan_command']))
                NewScanCommand = reduce_SC(ScanCommand)

                # Find the ranges of all the manipulator degrees of freedom
                # A List cannot be written to a cell, therefore make the list variable a string
                dof_dict = {}
                for dof_key in manipulator_keys:
                    dof = np.squeeze(f['entry1/instrument/manipulator/'+dof_key])
                    if dof.ndim == 0:
                        dof_range = sigfig(dof, 4)
                    else:
                        dof_range = str([sigfig(np.min(dof), 4), sigfig(np.max(dof), 4)])
                    dof_dict[dof_key] = dof_range

                # Other variables include deflector_x and energy (Photon energy), if these variables are
                # swept they will be within the analyser keys
                if 'deflector_x' in [*f['entry1/analyser'].keys()]:
                    dof = np.squeeze(f['entry1/analyser/deflector_x'])
                    dof_range = str([sigfig(np.min(dof), 4), sigfig(np.max(dof), 4)])
                elif 'deflector_x' in [*f['entry1/instrument/analyser'].keys()]:
                    dof_range = sigfig(np.squeeze(f['entry1/instrument/analyser/deflector_x']), 4) 
                else:
                    dof_range = 'n/a'
                dof_dict['deflector_x'] = dof_range

                if 'energy' in [*f['entry1/analyser'].keys()]:
                    dof = np.squeeze(f['entry1/analyser/energy'])
                    dof_range = str([sigfig(np.min(dof), 4), sigfig(np.max(dof), 4)])
                else:
                    dof_range = sigfig(np.squeeze(f['entry1/instrument/monochromator/energy']), 4)
                dof_dict['Ph En'] = dof_range
                
                # Determine whether the scan is Gold
                saz = dof_dict['saz']
                if type(saz) == np.float64:
                    if abs(saz) > 20:
                        NewScanCommand = 'Gold'

                # Find analyser mode, polarization and Temp
                encoding = 'utf-8'
                mode = str(f['entry1/instrument/analyser/acquisition_mode'][0], encoding)
                Pol = str(f['entry1/instrument/insertion_device/beam/final_polarisation_label'][0], encoding)
                if 'horizontal' in Pol:
                    Pol = 'LH'
                if 'vertical' in Pol:
                    Pol = 'LV'
                T = sigfig(np.squeeze(f['entry1/sample/cryostat_temperature'][:]), 3)
                if type(T) == np.ndarray:
                    T = str([min(T), max(T)])

                # Find Pass energy and Exit slit size
                PassE = f['entry1/instrument/analyser/pass_energy'][0]
                ESlit = f['entry1/instrument/monochromator/exit_slit_size'][0]
            
            else:
                # Check if either cameras are in the entry1 keys
                cameras = np.array(['esi1', 'esi2', 'esi3'])
                mask = [x in entry_keys for x in cameras]
                if any(mask):
                    camera = cameras[mask][0]
                    NewScanCommand = 'Picture with '+camera
                    mode = camera
                    dim = 2
                # If not, class the scan as negligible
                else:
                    NewScanCommand = 'No info from analyser'
                    mode = 'n/a'
                    dim = 'n/a'
                dof_dict = {'Ph En':'n/a', 'sapolar':'n/a', 'deflector_x':'n/a', 'saazimuth':'n/a',
                            'satilt':'n/a', 'sax':'n/a', 'say':'n/a', 'saz':'n/a'}
                Pol = 'n/a'
                T = 'n/a'
                PassE = 'n/a'
                ESlit = 'n/a'
        
        # Now write in all the data 
        ws.write('A'+str(rowIndex), fname)
        ws.write('B'+str(rowIndex), NewScanCommand)
        
        # Make 3 dimensional data
        if dim == 3:
            ws.write('C'+str(rowIndex), dim, bold)
        else:
            ws.write('C'+str(rowIndex), dim)
            
        ws.write('D'+str(rowIndex), mode)
        ws.write('E'+str(rowIndex), Pol)
        ws.write('F'+str(rowIndex), T)
        ws.write('G'+str(rowIndex), dof_dict['Ph En'])
        ws.write('H'+str(rowIndex), dof_dict['sapolar'])
        ws.write('I'+str(rowIndex), dof_dict['deflector_x'])
        ws.write('J'+str(rowIndex), dof_dict['saazimuth'])
        ws.write('K'+str(rowIndex), dof_dict['satilt'])
        ws.write('L'+str(rowIndex), dof_dict['sax'])
        ws.write('M'+str(rowIndex), dof_dict['say'])
        ws.write('N'+str(rowIndex), dof_dict['saz'])
        ws.write('O'+str(rowIndex), PassE)
        ws.write('P'+str(rowIndex), ESlit)

        rowIndex += 1
        print(rowIndex-2, 'files done')
    
# Format the Cells
ws.set_row(1, 18, bold)
ws.set_column('A:A', 15, centering)
ws.set_column('B:B', 50, centering)
ws.set_column('C:C', 6, centering)
ws.set_column('D:F', 10, centering)
ws.set_column('G:N', 15, centering)
ws.set_column('O:P', 10, centering)
    
wb.close()
print('All files done')

2 files done
3 files done
4 files done
5 files done
6 files done
7 files done
8 files done
9 files done
10 files done
11 files done
12 files done
13 files done
14 files done
15 files done
16 files done
17 files done
18 files done
19 files done
20 files done
21 files done
22 files done
23 files done
24 files done
25 files done
26 files done
27 files done
28 files done
29 files done
30 files done
31 files done
32 files done
33 files done
34 files done
35 files done
36 files done
37 files done
38 files done
39 files done
40 files done
41 files done
42 files done
43 files done
44 files done
45 files done
46 files done
47 files done
48 files done
49 files done
50 files done
51 files done
52 files done
53 files done
54 files done
55 files done
56 files done
57 files done
58 files done
59 files done
60 files done
61 files done
62 files done
63 files done
64 files done
65 files done
66 files done
67 files done
68 files done
69 files done
70 files done
71 files done
72 files done
73 files done


In [ ]:
wb.close()